# TF-IDF 및 textrank적용 코드 정리

## 호출 코드 및 라이브러리 모음

In [1]:
#호출코드모음
import pandas as pd
import os
#마운트
from google.colab import drive
drive.mount('/content/drive')
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
#단어 빈도셀 때
from collections import Counter
import string

Mounted at /content/drive


In [2]:
#1. Konlpy에서 Okt
!pip install konlpy
#2. konlpy 실행을 위한 자바어쩌구 다운
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#3. 필요한 설정
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
from konlpy.tag import Okt

okt = Okt()
okt.morphs("한국외국어대학교에서 안내드립니다. 오늘은 학교수업이 없습니다.", stem = True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 37.2 MB/s eta 0:00:00


['한국',
 '외국어',
 '대학교',
 '에서',
 '안',
 '내',
 '드리다',
 '.',
 '오늘',
 '은',
 '학교',
 '수업',
 '이',
 '없다',
 '.']

### 사용자 사전 고치기
- 사용자 사전 만들기 (colab)
https://sirzzang.github.io/ai/AI-konlpy-userdic/

In [ ]:
os.chdir('/usr/local/lib/python3.10/dist-packages/konlpy/java')
os.getcwd()

'/usr/local/lib/python3.10/dist-packages/konlpy/java'

In [ ]:
# 최초 실행이후 makedirs는 주석처리
os.makedirs('./temp_fix')
os.chdir('./temp_fix')
os.getcwd()
!jar xvf ../open-korean-text-2.1.0.jar

  created: META-INF/
 inflated: META-INF/MANIFEST.MF
  created: org/
  created: org/openkoreantext/
  created: org/openkoreantext/processor/
  created: org/openkoreantext/processor/normalizer/
  created: org/openkoreantext/processor/phrase_extractor/
  created: org/openkoreantext/processor/qa/
  created: org/openkoreantext/processor/stemmer/
  created: org/openkoreantext/processor/tokenizer/
  created: org/openkoreantext/processor/tools/
  created: org/openkoreantext/processor/util/
  created: org/openkoreantext/processor/util/adjective/
  created: org/openkoreantext/processor/util/adverb/
  created: org/openkoreantext/processor/util/auxiliary/
  created: org/openkoreantext/processor/util/freq/
  created: org/openkoreantext/processor/util/josa/
  created: org/openkoreantext/processor/util/noun/
  created: org/openkoreantext/processor/util/substantives/
  created: org/openkoreantext/processor/util/typos/
  created: org/openkoreantext/processor/util/verb/
 inflated: org/openkoreantext/pr

In [ ]:
# 사용자 사전 열기
with open(f"/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix/org/openkoreantext/processor/util/noun/names.txt") as f:
    data = f.read()

In [ ]:
#추가할 단어 입력
data += '한국외국어대학교\n한국외대\n안내\종합정보시스템\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n지원신청서\n수강료\n등록기간\n수강신청자\n한하다\n추가신청\n지원자\n선발자\n미수강신청자\n국제지역대학원\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\종합정보화면\n계절학기\n부분환불\n계절학기\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\n신종코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸업대기\nELLT학과\n영어학과\n아랍어과\n미디어커뮤니케이션학부\n영어전공\n영미문학문화학과\n태국어과\n경제학부\nEICC학과\n영어통번역학과\n베트남어과\n국제통상학과\이란어과\n경영학부\n프랑스어학부\n인도어과\n영어교육과\n독일어과\n터키아제르바이잔어과\n프랑스어교육과\n노어과\n몽골어과\n독일어교육과\n스페인어과\n중국언어문화학부\n한국어교육과\n이탈리아어과\n중국외교통상학부\n국제학부\n포르투갈어과\n그리스불가리아학과\n일본언어문화학부\n사회과학전공\n네덜란드어과\n융합일본지역학부\nLD전공\n스칸디나비아어과\n정치외교학과\nLT전공\n말레이인도네시아어과\n행정학과\n서양어대학\n아시아언어문화대학\n중국어대학\n일본어대학\n사회과학\n상경\n경영\n국제학부\n사회과학전공\n사범대학\n교직\n실용외국어\n군입대\n입영통지서\n미비\n미충족\n등록휴학\n휴학\n전역일\n증빙서류\n1학기\n2학기\n개강\n종강\n학사일정\n행사명\n조치사항\n참고사항\n공고\n부서명\n학년도\n개설학과\n학수번호\교과목명\시간\n교강사명\n강의시간\n폐강공고\n일정\n단과대\n배부처\n문의\n동유럽학대학\n국제지역대학\n어문학관\n통번역대학\n인문대학\n경상대학\n자연과학대학\n학생회관\n공과대학\n바이오메디컬공학부\n졸업증명서\n문의\n전화\문의전화\n원스톱서비스센터\n졸업예정증명서\n편입생\n신입생\n신편입생\n아래\nQR코드\n지양하다\n투명성\n공정성\n외국인교환학생\n외국인방문학생\n첫학기\n영어진단평가\n원격\n비대면\ne-Class\neclass\n이클래스\n안내하다\n전자출결\n전자출결시스템\n체계적인\n엄정성\n공고\n코로나19\n일반휴학\n신청\n별도문의\nCOVID-19\nCovid19\covid19\n중간시험\기말시험\n응시인원\nCommunicativeEnglish\nLS\nL/S\n우리대학\n콜로키움\nEU연구소\n인문사회연구사업단\n온·오프라인\n온오프라인\nHK+사업단\n국제관\n인도연구소\n패널티\n수습국원\n추가모집\n게시물\n외대교육방송국\n교수학습개발원\nFBS\주한브라질대사관\n세계여성의날\스마트도서관\n휠라아쿠쉬네트홀\n영화\n상영\n한국어자막\대학혁신지원사업\n아이디어톤\n수도권역\n미래대학투모로우솔루션\n미네르바헬스장\n교육생\n휴관\n금모닝\n글로벌캠퍼스\n누구나\n회차별\n보건실\n대한적십자사\n헌혈증\n헌혈자\n성격유형검사\n핵심역량통합관리시스템\nHufsability\n교과\n비교과\n서울학생상담센터\n추후공지\n청년몽땅정보통\n마이페이지\n문의처\n다산콜센터\특별전형\n일반전형\n면접전형\n등록예치금\n예치금\n모집\n학과\n전공\nTESOL대학원\n세미오시스연구센터\n서울학생상담센터\n전문\n경영대학원\n해당일\n접수마감일\n해당학기\n졸업자\n졸업예정자\n외국대학\n소지자\n한국어능력시험\n입증서류\n출신학교\n주한공관영사\n입증서류\n학사학위\n학사\n최종학력\n증명\n학위증서\n개인정보이용및제공동의서\n기타서류\n해당자\n추천서\n경력증명서\n외국국적지원자\n성적증명서\n총점\n사이버외대\n군인공제회직원\n학교교직원\n첫학기\n입학금\n운영위원회\n전형료\n우편접수\n입학원서\n본인날인\n서명함\n수험표\n지참\n제출서류\n외국어교육센터\nK-Zone\nNoshow\n출석차감\n1교시\n2교시\n3교시\n4교시\n5교시\n6교시\n7교시\n8교시\n9교시\n10교시\n11교시\n12교시\n작문\n대학영어\n중간시험\n보충수업\n추가모집\n창업자\nAI/SW\n모집공고\n모집분야\n일반창업\n스펙업\n졸업대기자\n수료생\nSW중심대학사업단\nAI교육원\nSW관련학과\n컴퓨터공학부\n정보통신공학부\nGBT전공\nELLT학과\nAI융합전공\n데이터사이언스전공\n바이오데이터사이언스전공\nICT&AI세부모듈\n수혜제외\n초과학기생\n초과학기\n9학기\n1학기\n2학기\n3학기\n4학기\n5학기\n6학기\n7학기\n8학기\n지원사항\n팀별\n개인별\n사업자등록증\n최종보고서\n학기말\n학기초\n전세계\n삼성그룹\nLG그룹\n주기적\n인증테스트\n비교과\n굿즈\n코로나19\n경제적\n지원\n타지역\n이에\n랜선나눔캠퍼스\n활동개요\n신청기간\n비합격\n비선발자\n최종합격자\n활동불가자\n활동혜택\n자기소개서\n1부\n학습계획서\n증빙자료\n재학증명서\n1차합격\n2차합격\n스캔\n활동비\소득세\n지방소득세\n문의처\n추후공지\n공공인재개발원\n개인사물함\n행정입법고시\n2차모의고사\n수험서적\n사민재\n세미나실\n국제학생회\n본교생\n외국인교환학생\n정기총회\n뵨교재학생\n버디프로그램\n홍보팀\n 기획컬처팀\n국제교류처\n국제교류처장\n불합격자\n코로나-19\n양성판정\n음성판정\n양성\n음성\n최종합격자\nQ&A\n필참\n퇴출\n정기활동\n이메일접수\n시험일정\n구분\n시간\n비고\n시험장소\n입실완료\n인문과학관\n졸업유예생\n학부생들\n코로나-19\n글로벌캠퍼스\n보건실\n학생부종합전형\n합격수기\n선배들\n글로벌창업지원단\n창업인재양성센터\n창업동아리\n미디어커뮤니케이션대학원\n대학원장\nCulture&Technology융합대학\n지원자격\n지원서\n접수\n평가요소\n영자신문사\nTheArgus\n수습기자\n디아거스\n공식홈페이지\n공식인스타그램\n사회봉사센터\n국제학사\n우편접수\n서울시동대문구이문로107\n대학안전관리계획\n붙임팡리\n안전보건관리팀\n동원번역상\n한국외국어대학교\n한국외대\n안내\종합정보시스템\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n수강료\n등록기간\n수강신청자\n한하다\n미수강신청자\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\종합정보화면\n계절학기\n부분환불\n계절학기\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\n신종코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n논문\n제적\n졸업요건\n지역학사\n지역전문가과정\n수료졸업대기\nELLT학과\n영어학과\n아랍어과\n미디어커뮤니케이션학부\n영어전공\n영미문학문화학과\n태국어과\n경제학부\nEICC학과\n영어통번역학과\n베트남어과\n국제통상학과\이란어과\n경영학부\n프랑스어학부\n인도어과\n영어교육과\n독일어과\n터키아제르바이잔어과\n프랑스어교육과\n노어과\n몽골어과\n독일어교육과\n스페인어과\n중국언어문화학부\n한국어교육과\n이탈리아어과\n중국외교통상학부\n국제학부\n포르투갈어과\n그리스불가리아학과\n일본언어문화학부\n사회과학전공\n네덜란드어과\n융합일본지역학부\nLD전공\n스칸디나비아어과\n정치외교학과\nLT전공\n말레이인도네시아어과\n행정학과\n서양어대학\n아시아언어문화대학\n중국어대학\n일본어대학\n사회과학\n상경\n경영\n국제학부\n사회과학전공\n사범대학\n교직\n실용외국어\n군입대\n입영통지서\n미비\n미충족\n등록휴학\n휴학\n전역일\n증빙서류\n1학기\n2학기\n개강\n종강\n학사일정\n행사명\n조치사항\n참고사항\n공고\n부서명\n학년도\n개설학과\n학수번호\교과목명\시간\n교강사명\n강의시간\n폐강공고\n일정\n단과대\n배부처\n문의\n동유럽학대학\n국제지역대학\n어문학관\n통번역대학\n인문대학\n경상대학\n자연과학대학\n학생회관\n공과대학\n바이오메디컬공학부\n졸업증명서\n문의\n전화\문의전화\부전공\n이중전공\n취소일정\n아래n원스톱서비스센터\n졸업예정증명서\n편입생\n신입생\n신편입생\n아래\nQR코드\n지양하다\n투명성\n공정성\n외국인교환학생\n외국인방문학생\n첫학기\n영어진단평가\n원격\n비대면\ne-Class\neclass\n이클래스\n안내하다\n전자출결\n전자출결시스템\n체계적인\n엄정성\n공고\n코로나19\n일반휴학\n신청\n별도문의\nCOVID-19\nCovid19\covid19\n중간시험\기말시험\n응시인원\nCommunicativeEnglish\nLS\nL/S\n우리대학\n기타사항\n부전공\n전공심화\n단일전공\n기존이중전공\n이중전공\n취소신청\n학사공지\n다음학기\n변경기간\코어사업\n학석사연계과정\n순수외국인\n사범대생\n불필요\n이수학점\n외국어로서의한국어교육전공\n외국어로서의한국어통번역전공\n입학전형\n불가\n신청불가\nHUFSService\n재입학\n현황\n전자규정집\n음성파일\n교수님\n추가선발\n신청자격\n학기기준\n등록생\n국내외교류\n인성검사\n교직적성\n사회적거리두기\n무효화\n대리신청\n수강신청\n학생인재개발처장\n정보지원처장\n추가선발\n이수예정자\n미신청\n미충원\n독일어통번역학과\n스페인어통번역학과\n아랍어통번역학과\n프랑스학과\n러시아학과\n국제금융학과\n전자물리학과\n교원자격증\n보강주간\nF학점\n중복수강\n동일교과목\n유사교과목\n복학예정자\n사범계열학생\n사범대학\nLT학부\nLD학부\nE-mail\n상경대학\n국제통상학과\n경제학부\n경영대학\n경영학부\n전입인원\n독일어교육전공\n중국어교육전공\nKFL학부\nLanguage&Diplomacy\nLanguage&Trade학부\n철학과\n사학과\n언어인지학과\n영어통번역학부\n독일어통번역학과\n스페인어통번역학과\n이탈리아어통번역학과\n중국어통번역학과\n일본어통번역학과\n말레이인도네시아어통번역학과\n태국어통번역학과\n동유럽대학\n폴란드어과\n루마니아어과\n체코·슬로바키아어과\n헝가리어과\n세르비아·크로아티아어과\n우크라이나어과\n프랑스학과\n브라질학과\n그리스·불가리아학과\n인도학과\n중앙아시아학과\n아프리카학부\n러시아학과\n한국학과\nGBT학부\n국제금융학과\n글로벌스포츠산업학부\n융합인재대학\n융합인재학부\n통계학과\n전자물리학과\n환경학과\n생명공학과\n화학과\n컴퓨터공학부\n컴퓨터공학과\n디지털정보공학과\n컴퓨터전자시스템공학부\n전자공학과\n산업경영공학과\n바이오메디컬공학부\n바이오메디컬공학전공\n'

with open("/usr/local/lib/python3.10/dist-packages/konlpy/java/temp_fix/org/openkoreantext/processor/util/noun/names.txt", 'w') as f:
    f.write(data)
data

'가몽\n가온\n갓세븐\n강새이\n게임닉가\n관우\n귀여미\n규\n김유이\n김준면\n까까런\n노컷\n누너예\n니노\n다마고치\n다이무스\n대학생\n데이브\n도요토미\n동운\n동이\n두주니\n디시인사이드\n디오\n라몹\n라스\n라옵\n멍구\n메이든\n명덕\n명량\n문민정부\n미네\n방엘리\n병헌\n붓다\n비정상회담\n빼빼로\n삼풍\n샤인온미\n성식\n성열\n세라문\n세라복\n세종대왕\n손권\n손책\n쇼미더머니\n쇼챔\n순규\n스라소니\n신동아\n신쓰패밀리\n신아라\n아베\n안상홍\n안홍준\n여누\n여랑\n여포\n연합\n오꾸닭\n요섭\n웃찾사\n원식\n유병언\n유비\n유이\n윤기형\n이나단\n이명박\n이완용\n임창용\n자괴\n자니윤\n자대련\n자유\n재중이\n전교조\n정윤회\n제갈량\n조자룡\n조조\n준면\n지오디\n지존파\n진영오\n차작가\n차트\n창섭\n챠트\n첸\n코르사주\n하무열\n하용파쿠\n혁재\n현이\n현태\n혜미\n한국외국어대학교\n한국외대\n안내\\종합정보시스템\\공고\n이수\n학기\n대하여\n아래\n수강편람\n이중전공\n수강신청함\n수강신청\n예비수강신청\n교무처장\n지원신청서\n수강료\n등록기간\n수강신청자\n한하다\n추가신청\n지원자\n선발자\n미수강신청자\n국제지역대학원\n납부불가\n핫링크\n고지서출력\n가상계좌\n붙임파일\n모바일앱\n내역\n인터넷뱅킹\n텔레뱅킹\n개시일\n신청\n마감\n마감일\n수강취소\n수강변경\\종합정보화면\n계절학기\n부분환불\n계절학기\\학년도\n접수일\n학석사\n학사\n석사\n연계\n과정생\n접수처\n학사종합지원센터\n양캠퍼스\n통번역\n이수\n미배정\n부전공\\여석\n이수\n지원\n학사종합지원센터\n한국예술종합학교\n특성화\n수강편람\n실용외국어\n공동교양\n여석\\졸업\n사정\n결과\n확정자\n불가자\n개강일\n우리대학교\n우리대학\n안내문\n학위\n학위증\n코로나바이러스\n신종코로나바이러스\nHUFS\n양캠퍼스\n서울캠퍼스\n글로벌캠퍼스\n서울글로벌캠퍼스\n서울·글로벌캠퍼스\n우리은행\n단과대학\n졸업시험\n

In [ ]:
!jar cvf ../open-korean-text-2.1.0.jar *

added manifest
ignoring entry META-INF/
adding: META-INF/maven/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/(in = 0) (out= 0)(stored 0%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.xml(in = 9127) (out= 2208)(deflated 75%)
adding: META-INF/maven/org.openkoreantext/open-korean-text/pom.properties(in = 119) (out= 110)(deflated 7%)
ignoring entry META-INF/MANIFEST.MF
adding: org/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/OpenKoreanTextProcessor$.class(in = 7716) (out= 2400)(deflated 68%)
adding: org/openkoreantext/processor/phrase_extractor/(in = 0) (out= 0)(stored 0%)
adding: org/openkoreantext/processor/phrase_extractor/KoreanPhraseExtractor.class(in = 6005) (out= 3967)(deflated 33%)
adding: org/openkoreantext/proc

In [ ]:
#세션 재시작후 작동 확인
import os
from konlpy.tag import Okt
new_dir = '/content'

# 작업 디렉토리 변경
os.chdir(new_dir)
okt = Okt()
print(okt.nouns('송이레만세')) # ['송이레만세']
print(okt.morphs('김재경자경')) # ['김재경자경']
print(okt.morphs("한국외국어대학교"))

['송이', '레만', '세']
['김재경', '자', '경']
['한국외국어대학교']


### 각종 문서 저장 및 copy

In [3]:
#각종 문서 저장

gongji = pd.read_csv('./drive/MyDrive/csv모음/final_merged_gongji(0501).csv')
haksa = pd.read_csv('./drive/MyDrive/csv모음/merged_haksa.csv')
hiring = pd.read_csv('./drive/MyDrive/csv모음/merged_hire.csv')
janghak = pd.read_csv('./drive/MyDrive/csv모음/merged_janghak.csv')

#공지 분할 저장
gongji_1 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_1.csv')
gongji_2 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_2.csv')
gongji_3 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_3.csv')
gongji_4 = pd.read_csv('./drive/MyDrive/csv모음/final_gongji_4.csv')

In [4]:
#파일복사
# 원본 데이터 보존 위해 copy 데이터 생성
gongji_copy = gongji.copy()
haksa_copy = haksa.copy()
hiring_copy = hiring.copy()
janghak_copy = janghak.copy()


gongji1_copy = gongji_1.copy()
gongji2_copy = gongji_2.copy()
gongji3_copy = gongji_3.copy()
gongji4_copy = gongji_4.copy()

## 공지 처리

### 공지1

## 학사 처리

In [ ]:
haksa_copy

,index,date,title,content
0,1,2020-01-06,[공통] 2019-2학기 이중전공 변경 배정 확정 공고,\n\n\n\n\n\n[공통]이중전공변경 배정 확정 공고\n \n2019학년도 제2...
1,2,2020-01-13,[공통] 2020학년도 1학기 수강신청 공고,\n2020학년도 1학기 수강신청 공고 2020학년도 제1학기 수강신청에 대하여 아...
2,3,2020-01-13,[공통] 2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내,\n2020학년도 전기 국제지역대학원 학석사연계과정 모집 안내\n \n2020학년도...
3,4,2020-01-13,[공통] 2020학년도 전기 일반대학원 학석사연계과정 모집 안내,\n2020학년도 전기 일반대학원 학석사연계과정 모집 안내\n\n2020학년도 전기...
4,5,2020-01-15,[글로벌]졸업 예비사정 결과 조회,\n[글로벌] 졸업 예비사정 결과 조회글로벌캠퍼스 졸업(2020년 2월) 예비사정 ...
...,...,...,...,...
617,618,2024-03-13,[공통] 생리 유고 신청 안내,\n\n\n\n\n\n생리 유고 신청 안내\n\n\n\n\n\n 생리로 인한 유고...
618,619,2024-03-13,[공통] 졸업예정자(해당학기 졸업가능자)의 조기취업 유고결석 신청 안내,\n\n\n\n\n\n졸업예정자(해당학기 졸업가능자)의 조기취업유고결석 신청 안내\...
619,621,2024-03-15,[글로벌]2024-1학기 폐강(2차) 공고(글로벌캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(글로벌캠퍼스)\n\n\n\n...
620,620,2024-03-15,[서울] 2024-1학기 폐강(2차) 공고(서울캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(서울캠퍼스)\n\n\n\n\...


In [ ]:
# 코로나 count -> 6개라 안지우고 일단 해봄
corona_count = len(haksa_copy[haksa_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(haksa_copy))

코로나19가 포함된 데이터 개수: 6
삭제 전 데이터 개수: 622


In [ ]:
def preprocess_haksa(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    text= re.sub(r'\n','',text)
    return text

# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
haksa_copy['cleaned_content'] = haksa_copy['content'].apply(preprocess_haksa)
haksa_copy.tail()

,index,date,title,content,cleaned_content
617,618,2024-03-13,[공통] 생리 유고 신청 안내,\n\n\n\n\n\n생리 유고 신청 안내\n\n\n\n\n\n 생리로 인한 유고...,생리 유고 신청 안내 생리로 인한 유고결석 신청 방법을 아래와 같이 안내합니다- ...
618,619,2024-03-13,[공통] 졸업예정자(해당학기 졸업가능자)의 조기취업 유고결석 신청 안내,\n\n\n\n\n\n졸업예정자(해당학기 졸업가능자)의 조기취업유고결석 신청 안내\...,졸업예정자해당학기 졸업가능자의 조기취업유고결석 신청 안내졸업예정자해당학기 졸업가능자...
619,621,2024-03-15,[글로벌]2024-1학기 폐강(2차) 공고(글로벌캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(글로벌캠퍼스)\n\n\n\n...,2024-1학기 폐강2차 공고글로벌캠퍼스2024-1학기 글로벌캠퍼스 폐강2차 강좌를...
620,620,2024-03-15,[서울] 2024-1학기 폐강(2차) 공고(서울캠퍼스),\n\n\n\n\n\n2024-1학기 폐강(2차) 공고(서울캠퍼스)\n\n\n\n\...,2024-1학기 폐강2차 공고서울캠퍼스2024학년도 제1학기 서울캠퍼스 폐강2차 강...
621,622,2024-03-26,[공통]2024-여름계절학기 개설희망 교과목 수요조사 실시 공고,\n계절학기 개설희망 교과목 수요조사 실시 공고\n\n2024학년도여름계절학기 개설...,계절학기 개설희망 교과목 수요조사 실시 공고2024학년도여름계절학기 개설희망 교과목...


In [ ]:
'''#토큰화: 사용자 사전 추가시 사용
def tokenize_sentences(sentence):
    tokens = okt.pos(sentence)  # 형태소 분석 및 품사 태깅
    return tokens

# 'cleaned_content' 열에 있는 각 문장에 토큰화 및 품사 태깅 적용
haksa_copy['tagged_content'] = haksa_copy['cleaned_content'].apply(tokenize_sentences)
# 결과 확인
print(haksa_copy['tagged_content'])'''

"#토큰화: 사용자 사전 추가시 사용\ndef tokenize_sentences(sentence):\n    tokens = okt.pos(sentence)  # 형태소 분석 및 품사 태깅\n    return tokens\n\n# 'cleaned_content' 열에 있는 각 문장에 토큰화 및 품사 태깅 적용\nhaksa_copy['tagged_content'] = haksa_copy['cleaned_content'].apply(tokenize_sentences)\n# 결과 확인\nprint(haksa_copy['tagged_content'])"

In [ ]:
okt = Okt()
words = " ".join(haksa_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)
#28만개 정도
len(words)

285758

In [ ]:
# 형태소 분석기 초기화
okt = Okt()

# 텍스트에서 명사만 추출
nouns = "".join(haksa_copy['cleaned_content'].tolist())
nouns = okt.nouns(nouns)

In [ ]:
#16만개
len(nouns)

167926

In [ ]:
#한글자지우기 -> 14만개
remove_nouns =  [x for x in nouns if len(x)>1 and len(x) < 15]
len(remove_nouns)

145179

In [ ]:
from collections import Counter
frequent = Counter(remove_nouns).most_common()
frequent

[('신청', 3259),
 ('전공', 2613),
 ('학기', 2456),
 ('학과', 1996),
 ('학생', 1767),
 ('학점', 1711),
 ('수강신청', 1639),
 ('대학', 1556),
 ('이중전공', 1515),
 ('경우', 1478),
 ('수강', 1415),
 ('변경', 1403),
 ('이수', 1181),
 ('졸업', 1173),
 ('가능', 1112),
 ('부전공', 1079),
 ('확인', 1064),
 ('기간', 1045),
 ('불가', 1022),
 ('과목', 992),
 ('제출', 988),
 ('납부', 983),
 ('수업', 963),
 ('배정', 818),
 ('해당', 811),
 ('성적', 800),
 ('학교', 783),
 ('교과목', 777),
 ('학년도', 773),
 ('글로벌캠퍼스', 765),
 ('취소', 743),
 ('종합정보시스템', 697),
 ('글로벌', 696),
 ('학년', 680),
 ('일정', 677),
 ('서울캠퍼스', 676),
 ('학번', 676),
 ('안내', 675),
 ('서울', 641),
 ('교양', 634),
 ('사항', 623),
 ('아래', 617),
 ('등록', 615),
 ('학사종합지원센터', 612),
 ('이상', 608),
 ('인정', 594),
 ('휴학', 593),
 ('등록금', 582),
 ('홈페이지', 567),
 ('공고', 565),
 ('이후', 559),
 ('방법', 521),
 ('재수강', 519),
 ('계절학기', 517),
 ('반드시', 516),
 ('기준', 500),
 ('캠퍼스', 500),
 ('문의', 489),
 ('희망', 478),
 ('학부', 478),
 ('시험', 460),
 ('예정', 459),
 ('관련', 455),
 ('교무처장', 449),
 ('교류', 446),
 ('본인', 437),
 ('시간', 430),
 ('교직',

In [ ]:
#불용어 처리
with open('./stopwords_haksa.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords_nouns = [x for x in remove_nouns if x not in stopwords]
#13만개
len(remove_stopwords_nouns)

138981

In [ ]:
#불용어 처리 후 빈도확인
Counter(remove_stopwords_nouns).most_common()

In [ ]:
# remove_stopwords 리스트를 Counter로 변환하여 가장 적게 나오는 순서대로 출력
least_common_words = Counter(remove_stopwords_nouns).most_common()[::-1]

# 결과 출력
print(least_common_words)

[('한국인', 1), ('권지은', 1), ('신경', 1), ('신경언어학', 1), ('스칸디나비아', 1), ('노르웨이', 1), ('붕익', 1), ('김다정', 1), ('김정민', 1), ('용어', 1), ('문맥', 1), ('김수완', 1), ('김정숙', 1), ('전통', 1), ('이송', 1), ('박재윤', 1), ('난민', 1), ('윤태중', 1), ('상목', 1), ('유명', 1), ('대중', 1), ('김인', 1), ('연진', 1), ('안목', 1), ('호티롱', 1), ('전미라', 1), ('구조', 1), ('윤희수', 1), ('신영수', 1), ('연금', 1), ('김준', 1), ('파이썬', 1), ('불가리아인', 1), ('혜월', 1), ('홍병', 1), ('양수', 1), ('국제경제', 1), ('권한', 1), ('무단', 1), ('이상은', 1), ('표절', 1), ('다한', 1), ('달성', 1), ('학업성취도', 1), ('번더', 1), ('우리나라', 1), ('정규직', 1), ('아르바이트', 1), ('창업자', 1), ('프리랜서', 1), ('직계존속', 1), ('고결', 1), ('여학생', 1), ('대체휴일', 1), ('검색', 1), ('차로', 1), ('거짓', 1), ('전송', 1), ('노자', 1), ('페강', 1), ('트리', 1), ('계삲함', 1), ('유라시아', 1), ('미술사', 1), ('독성학', 1), ('금융기관', 1), ('와글', 1), ('아세안', 1), ('태국', 1), ('말함', 1), ('해국', 1), ('이용기', 1), ('거미', 1), ('관해', 1), ('호석', 1), ('연관', 1), ('관자', 1), ('동선', 1), ('외고', 1), ('점심시간', 1), ('졸업대기자', 1), ('적미', 1), ('사예', 1), ('의비', 1), ('배정자', 1), ('답글

In [ ]:
nouns = remove_stopwords_nouns
len(nouns)

138981

### 학사 TF-IDF 적용

In [ ]:
'''#학사 정제된 명사들 다운
from google.colab import files

# 파일을 쓰기 모드로 열고 데이터를 쓴 후 파일을 다운로드합니다.
with open('haksa_nouns_list.txt', 'w', encoding='utf-8') as file:
    for noun in nouns:
        line = ''.join(noun)
        file.write(line + '\n')

# 다운로드할 파일을 지정합니다.
files.download('haksa_nouns_list.txt'''')

In [ ]:
#명사 파일열기
with open('haksa_nouns_list.txt','r',encoding = 'utf-8') as file:
  for _ in range(5):
    line = file.readline()
    print(line)

공통

이중전공

변경

배정

확정



In [ ]:
#tf-idf 넣어줄 사이킷런다운
!pip install -U scikit-learn
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import numpy as np
from scipy.sparse import csr_matrix
from konlpy.tag import Okt

tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []


In [ ]:
#명사읽어오기
with open ("haksa_nouns_list.txt", 'r', encoding = 'utf-8') as file:
  lines = file.readlines()
  data = [line.strip() for line in lines]
haksa_nouns = data
haksa_nouns

['공통',
 '이중전공',
 '변경',
 '배정',
 '확정',
 '공고',
 '학년도',
 '학기',
 '이중전공',
 '변경',
 '배정',
 '결과',
 '아래',
 '공고함',
 '결과',
 '조회',
 '종합정보시스템',
 '전공',
 '교직',
 '관리',
 '이중전공',
 '변경',
 '신청',
 '안내',
 '사항',
 '이중전공',
 '변경',
 '배정',
 '학생',
 '이중전공',
 '변경',
 '신청',
 '불가',
 '미배정',
 '학생',
 '기존',
 '수중',
 '이중전공',
 '부전공',
 '유지',
 '이중전공',
 '변경',
 '기간',
 '신청',
 '가능',
 '졸업예정자',
 '학년',
 '수료',
 '졸업',
 '대기',
 '포함',
 '구분',
 '변경',
 '신청',
 '영역',
 '졸업',
 '학점',
 '누락',
 '이중전공',
 '변경',
 '성적',
 '조회',
 '변경',
 '구분',
 '수정',
 '학사종합지원센터',
 '전공',
 '소속',
 '단과대학',
 '창구',
 '방문',
 '구분',
 '변경',
 '신청',
 '작성',
 '제출',
 '방학',
 '학기',
 '가능',
 '기타사항',
 '부전공',
 '전공심화',
 '부전공',
 '단일전공',
 '기존',
 '이중전공',
 '부전공',
 '취소',
 '신청기간',
 '종합정보시스템',
 '신청',
 '중순',
 '학사공지',
 '바람',
 '학기',
 '이중전공',
 '변경',
 '기간',
 '예정',
 '교무처장',
 '학년도',
 '학기',
 '수강신청',
 '공고',
 '학년도',
 '학기',
 '수강신청',
 '아래',
 '공고',
 '강의',
 '시간표',
 '강의',
 '계획',
 '조회',
 '수강편람',
 '열람',
 '강의',
 '시간표',
 '강의',
 '계획',
 '조회',
 '학교',
 '홈페이지',
 '강의',
 '시간표',
 '조회',
 '변동',
 '사항',
 '실시간',
 '반영',
 '수강신청',
 '확인',


In [ ]:
def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()

    return tfidf_mat

sent_graph = build_sent_graph(haksa_nouns)
sent_graph

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# tfidf_mat를 csr_matrix로 변환
tfidf_mat_sparse = csr_matrix(sent_graph)

# 전치행렬 계산
tfidf_mat_transpose = tfidf_mat_sparse.T

# 전치행렬 곱 계산
graph_sentence_sparse = tfidf_mat_sparse.dot(tfidf_mat_transpose)

# 희소 행렬의 일부를 샘플링하여 출력
print(graph_sentence_sparse[100:105, 100:105].toarray())

[[1. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 1.]]


In [ ]:
# 13만8천개 정도의 정방행렬
graph_sentence_sparse.shape

(138981, 138981)

In [ ]:
#words graph
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer

def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

words_graph, idx2word = build_words_graph(haksa_nouns)

In [ ]:
words_graph

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
idx2word

{240: '공통',
 2295: '이중전공',
 1182: '변경',
 1140: '배정',
 3471: '확정',
 212: '공고',
 3249: '학년도',
 3245: '학기',
 140: '결과',
 1800: '아래',
 213: '공고함',
 2691: '조회',
 2715: '종합정보시스템',
 2533: '전공',
 316: '교직',
 267: '관리',
 1757: '신청',
 1818: '안내',
 1373: '사항',
 3263: '학생',
 1285: '불가',
 1021: '미배정',
 467: '기존',
 1641: '수중',
 1265: '부전공',
 2148: '유지',
 444: '기간',
 3: '가능',
 2701: '졸업예정자',
 3248: '학년',
 1614: '수료',
 2695: '졸업',
 691: '대기',
 3184: '포함',
 331: '구분',
 1964: '영역',
 3286: '학점',
 645: '누락',
 1526: '성적',
 1637: '수정',
 3261: '학사종합지원센터',
 1561: '소속',
 666: '단과대학',
 2883: '창구',
 1119: '방문',
 2466: '작성',
 2675: '제출',
 1130: '방학',
 471: '기타사항',
 2534: '전공심화',
 669: '단일전공',
 3017: '취소',
 1758: '신청기간',
 2772: '중순',
 3257: '학사공지',
 1053: '바람',
 1980: '예정',
 285: '교무처장',
 1593: '수강신청',
 71: '강의',
 1697: '시간표',
 184: '계획',
 1597: '수강편람',
 1941: '열람',
 3240: '학교',
 3432: '홈페이지',
 1183: '변동',
 1774: '실시간',
 1102: '반영',
 3469: '확인',
 129: '게시',
 2389: '일정',
 2476: '장바구니',
 941: '명칭',
 1973: '예비수강신청',


In [ ]:
def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

In [ ]:
haksa_rank_idx = get_ranks(graph_sentence_sparse)
haksa_rank_idx

In [ ]:
haksa_rank_idx

In [ ]:
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"="*44}')

Training MNIST Model on cuda


## 장학처리

In [ ]:
janghak_copy

In [ ]:
def preprocesss_haksa(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    text= re.sub(r'\n','',text)
    return text

# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
janghak_copy['cleaned_content'] = janghak_copy['content'].apply(preprocesss_haksa)
janghak_copy.tail()

In [ ]:
'''#토큰화: 사용자 사전 추가시 사용
def tokenize_sentences(sentence):
    tokens = okt.pos(sentence)  # 형태소 분석 및 품사 태깅
    return tokens

# 'cleaned_content' 열에 있는 각 문장에 토큰화 및 품사 태깅 적용
janghak_copy['tagged_content'] = janghak_copy['cleaned_content'].apply(tokenize_sentences)
# 결과 확인
print(janghak_copy['tagged_content'])'''

In [ ]:
okt = Okt()
words = " ".join(janghak_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)
#장학: 38만개-> 39만개 ->
len(words)

In [ ]:
# 형태소 분석기 초기화
okt = Okt()

# 텍스트에서 명사만 추출
nouns = "".join(janghak_copy['cleaned_content'].tolist())
nouns = okt.nouns(nouns)
#25만개
len(nouns)

In [ ]:
#한글자지우기 -> 14만개
remove_nouns =  [x for x in nouns if len(x)>1 and len(x) < 15]
#22만개
len(remove_nouns)

In [ ]:
from collections import Counter
frequent = Counter(remove_nouns).most_common()
frequent

In [ ]:
#불용어 처리
with open('./stopwords_janghak.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords_nouns = [x for x in remove_nouns if x not in stopwords]
#20만개
len(remove_stopwords_nouns)

In [ ]:
#불용어 처리 후 빈도확인
Counter(remove_stopwords_nouns).most_common()

## 채용 처리

In [ ]:
hiring_copy

In [ ]:
# 코로나 count
corona_count = len(hiring_copy[hiring_copy['title'].str.contains('코로나19')])
# 결과 출력
print("코로나19가 포함된 데이터 개수:", corona_count)
print("삭제 전 데이터 개수:", len(hiring_copy))

In [ ]:
def preprocess_haksa(text):
    # 링크 제거 (http:// 또는 https:// 로 시작하는 링크)
    text = re.sub(r'https?://\S+', '', text)
    # Zoom 회의 참가 ID 제거 _ 작동안함
    text = re.sub(r'Zoom 회의 참가 ID \d{3} \d{3} \d{4}', '', text)
    # 전화번호 제거 (괄호 선택적으로)
    text = re.sub(r'\(?0\d{2,3}\)?[- ]?\d{3,4}[- ]?\d{4}', '', text)
    # 전번 제거
    text = re.sub(r'\d{2}-\d{4}-\d{4}', '', text)
    # 팩스번호 제거
    text = re.sub(r'FAX\.\d{2}-\d{3}-\d{4}', '', text)
    # 주소 제거
    text = re.sub(r'서울시 동대문구 이문로 107', '', text)
    # 이메일 제거
    text = re.sub(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', '', text)
    # www 링크 제거
    text = re.sub(r'\bwww\.\S*', '', text)
    # hufs.ac.kr 링크 제거
    text = re.sub(r'hufs\.ac\.kr', '', text)
    #마지막 날짜 양식 변형
    text = re.sub(r'\d{4}\.\s\d{2}\.\s\d{2}\.', '', text)
    #마지막 날짜
    text = re.sub(r'\d{4}\.\s\d{2}\.', '', text) #과제 제출등에 유의
    # 문장부호 제거
    text = re.sub(r'[,.:!?\\|※‘’”“→-○☏『』)(]', '', text)
    text= re.sub(r'\n','',text)
    return text

# 'content' 열에 전처리 함수를 적용하여 새로운 열 생성
hiring_copy['cleaned_content'] = hiring_copy['content'].apply(preprocess_haksa)
hiring_copy.tail()

In [ ]:
okt = Okt()
words = " ".join(hiring_copy['cleaned_content'].tolist())
words = okt.morphs(words,stem=True)
#47만개 정도
len(words)

In [ ]:
from collections import Counter
frequent = Counter(words).most_common()
frequent

In [ ]:
# 형태소 분석기 초기화
okt = Okt()

# 텍스트에서 명사만 추출
nouns = "".join(hiring_copy['cleaned_content'].tolist())
nouns = okt.nouns(nouns)
#29만개
len(nouns)

In [ ]:
#한글자지우기 -> 24만개
remove_nouns =  [x for x in nouns if len(x)>1 and len(x) < 15]
#24만개
len(remove_nouns)

In [ ]:
#불용어 처리
with open('./stopwords_hiring.txt', 'r') as f:
    list_file = f.readlines()
stopwords = list_file[0].split(",")
remove_stopwords_nouns = [x for x in remove_nouns if x not in stopwords]
#19만개
len(remove_stopwords_nouns)

In [ ]:
nouns = remove_stopwords_nouns
#19만개
len(nouns)

In [17]:
a = hiring_copy.loc[2,'content']
type(a)

str

In [14]:
# 텍스트에서 명사만 추출
t_nouns = "".join(hiring_copy['content'].tolist())
type(t_nouns)

str

In [18]:
a = okt.nouns(a)
len(a)

215

In [11]:
# 형태소 분석기 초기화
okt = Okt()

# 텍스트에서 명사만 추출
nouns = "".join(hiring_copy['cleaned_content'].tolist())
nouns = okt.nouns(nouns)
#29만개
len(nouns)

AttributeError: 'str' object has no attribute 'a'